<h2>IMPORTS</h2>

In [ ]:
import os
import shutil

import matplotlib.pyplot as plt
import matplotlib.image as mpimg

import torch
import torchvision
import tempfile
import random
from tqdm import tqdm
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

import torch
from __future__ import division
import numpy as np
import math
import matplotlib.pyplot as plt
from tqdm import tqdm
from matplotlib import animation
from IPython.display import clear_output
import pandas as pd

from PIL import Image
from torchvision import datasets, transforms

import warnings
warnings.filterwarnings('ignore')

<h2>SIMULATIONS FUNCTIONS</h2>

In [ ]:
def linspace(start, stop, n):
    arr = []
    h = (stop - start) / (n - 1)
    for i in range(n):
        arr.append(start + (h * i))
    return arr

In [ ]:
''' Physical and temperature distribution parameters '''

Lx = 10e-6
dx = 50e-9
h0 = 100e-9
dt_default = 1000e-8
dt_scaleFactor = 0.5

p = 1
mode = 1
width = np.array(linspace((Lx/10), (Lx/4), 20))
amp = np.array(linspace(1,50,12))
shift = np.array(linspace(0, Lx, 10))
T_amb = 300

build_plots = False

In [ ]:
''' Material properties '''

mu = 2.5e4
B = -0.0855

In [ ]:
''' Simulation params '''

num_iterations = 4000
num_tests_to_complete = 1
n_cell = Lx/dx
n = int(n_cell + 1)

start = 0
xs = []
for i in range(int(Lx/dx)):
    xs.append(start)
    start += dx
xs = np.array(xs)

test_count = 0

i_max = len(width)
j_max = len(amp)

<h2>ARGUMENTS</h2>

In [ ]:
root_dir = 'datasets/MNIST_scale/'

path_list = []

for folder in os.listdir(root_dir):
    for sub_folder in os.listdir(root_dir + folder + '/'):
        for test_train_val_folder in os.listdir(root_dir + folder + '/' + sub_folder + '/'):
            for number_file in os.listdir(root_dir + folder + '/' + sub_folder + '/' + test_train_val_folder + '/'):
                for mnist_file in os.listdir(root_dir + folder + '/' + sub_folder + '/' + test_train_val_folder + '/' + number_file + '/'):
                    # print(root_dir + folder + '/' + sub_folder + '/' + test_train_val_folder + '/' + number_file + '/' + mnist_file)
                    path_list.append(root_dir + folder + '/' + sub_folder + '/' + test_train_val_folder + '/' + number_file + '/' + mnist_file)

<h2>GENERATE DATASET</h2>

In [ ]:
simulated_dataset_path = 'mnist_simulated_dataset/'

for path in path_list:
    print(path)
    index_file = path.split('/')[-1]
    seed_value = path.split('/')[-5].split('_')[-1]
    index_value = index_file.split('.')[0]
    new_file_name = 'T_' + seed_value + '_' + index_file
    new_path = simulated_dataset_path + 'png/' + new_file_name
    print('New path: ', new_path)
    shutil.copy(path, new_path)
    # simulate_tfle(path)

<h2>PNG -> PT</h2>

In [ ]:
for i in os.listdir('mnist_simulated_dataset/png/'):
    new_path = 'mnist_simulated_dataset/png/' + i
    
    if('.png' in i):
        image = Image.open(new_path)
        transform = transforms.Compose([
            transforms.Grayscale(),
            transforms.Resize(200),
            transforms.ToTensor(),
        ])
        
        img_tensor = transform(image)
        # print(i.split('.')[0] + '.pt')
        file_name = 'mnist_simulated_dataset/pt/' + i.split('.')[0] + '.pt'
        # print(file_name)
        torch.save(img_tensor, file_name)

<h2>CARRY OUT TFLE SIMULATIONS</h2>

In [ ]:
mnist_tensors_resized = []

tensor_300 = torch.empty(1, 200, 200).fill_(300.)

for tensor_file in os.listdir('mnist_simulated_dataset/pt/'):
    
    t_tensor = torch.load('mnist_simulated_dataset/pt/' + tensor_file)
    
    t_tensor = t_tensor * 30
    
    t_tensor = t_tensor + tensor_300
    
    mnist_tensors_resized.append(t_tensor)    

In [ ]:
mnist_tensors_resized = mnist_tensors_resized[:400]

In [ ]:
mnist_numpy_arrays = []

for tensor in mnist_tensors_resized:
    mnist_numpy_arrays.append(tensor.cpu().detach().numpy())  

In [ ]:
file_count = 0

In [ ]:
for mnist in tqdm(mnist_numpy_arrays):
    
    dt = dt_default
    test_count = i
    
    # t, amp_avg, amp1, amp2, w1, w2, shift1, shift2 = initialize_temperature_array(amp, width, shift, xs, T_amb, Lx, n)
    # t = initialize_temperature_array(amp, width, shift, xs, T_amb, Lx, n)
    
    mnist = mnist[0][:200, :200]
    
    # T = np.zeros((200, 200))
    # for j in range(200):
    #     T[j] = np.roll(t, -j)
        
    # T = np.subtract(T + mnist, 300)

    T = mnist
    
    T_x1 = np.roll(T, 1, axis = 1)
    Tx1 = np.roll(T, -1, axis = 1)
    T_y1 = np.roll(T, 1, axis = 0)
    Ty1 = np.roll(T, -1, axis = 0)
    
    dtdx = ((-0.5 * T_x1) + (0.5 * Tx1))/dx;
    dy = dx
    dtdy = ((-0.5 * T_y1) + (0.5 * Ty1))/dy;
    
    gam = 31.53 - (0.0855*T)
    
    gam_x1 = np.roll(gam, 1, axis = 1)
    gamx1 = np.roll(gam, -1, axis = 1)
    gam_y1 = np.roll(gam, 1, axis = 0)
    gamy1 = np.roll(gam, -1, axis = 0)
    
    # if(build_plots):
    #     plt.imshow(T, cmap = 'hot')
    
    for tr in range(1):
        if(tr == 0):
            dt = dt * dt_scaleFactor
        
        fluxX = np.zeros((200, 200))
        fluxY = np.zeros((200, 200))
        dfluxXdx = np.zeros((200, 200))
        dfluxYdy = np.zeros((200, 200))
        dhdt_max = 0
        h = h0 * np.ones((200, 200))
        status = 'incomplete'
        
        for r in range(num_iterations):
        # for r in range(10):
            # print('Enter num_iterations: ', r)
            
            h_x1 = np.roll(h, 1, axis = 1)
            hx1 = np.roll(h, -1, axis = 1)
            h_y1 = np.roll(h, 1, axis = 0)
            hy1 = np.roll(h, -1, axis = 0)
            
            h_x2 = np.roll(h, 2, axis = 1)
            hx2 = np.roll(h, -2, axis = 1)
            h_y2 = np.roll(h, 2, axis = 0)
            hy2 = np.roll(h, -2, axis = 0)
            
            fluxX_x1 = np.roll(fluxX, 1, axis = 1)
            fluxXx1 = np.roll(fluxX, -1, axis = 1)
            dfluxXdx_x1 = np.roll(dfluxXdx, 1, axis = 1)
            dfluxXdxx1 = np.roll(dfluxXdx, -1, axis = 1)
            
            fluxY_y1 = np.roll(fluxY, 1, axis = 0)
            fluxYy1 = np.roll(fluxY, -1, axis = 0)
            dfluxY_y1 = np.roll(dfluxYdy, 1, axis = 0)
            dfluxYy1 = np.roll(dfluxYdy, -1, axis = 0)
            
            dhdx2 = (h_x1 - (2 * h) + hx1)/(dx*dx)
            dhdx3 = (((-0.5 * h_x2) + h_x1 - hx1 + (0.5 * hx2))/(dx*dx*dx))
            
            dhdy2 = (h_y1 - (2 * h) + hy1)/(dy*dy)
            dhdy3 = (((-0.5 * h_y2) + h_y1 - hy1 + (0.5 * hy2))/(dy*dy*dy))
            
            deltah = dhdx2 + dhdy2
            
            deltahx1 = np.roll(deltah, -1, axis = 1)
            deltahy1 = np.roll(deltah, -1, axis = 0)
            deltah_x1 = np.roll(deltah, 1, axis = 1)
            deltah_y1 = np.roll(deltah, 1, axis = 0)
            
            ddeltahdx = ((-0.5*deltah_x1) + (0.5*deltahx1))/dx
            ddeltahdy = ((-0.5*deltah_y1) + (0.5*deltahy1))/dy
            
            fluxX = np.multiply(((np.square(h)*B)/(2*mu)), dtdx) + np.multiply(np.multiply((np.power(h, 3)/(3*mu)), dtdx), deltah) + np.multiply(np.multiply((np.power(h, 3)/(3*mu)), gam), ddeltahdx)
            fluxY = np.multiply(((np.square(h)*B)/(2*mu)), dtdy) + np.multiply(np.multiply((np.power(h, 3)/(3*mu)), dtdy), deltah) + np.multiply(np.multiply((np.power(h, 3)/(3*mu)), gam), ddeltahdy)
            
            dfluxXdx = ((-0.5 * fluxX_x1) + (0.5 * fluxXx1))/dx
            dflux = ((-0.5 * fluxY_y1) + (0.5 * fluxYy1))/dy
            
            dhdt = (-1 * dfluxXdx) + (-1 * dfluxYdy)
            
            if(dhdt_max > 10000000):
                status = 'failed'
                print('Failed')
                break
                
            if(dhdt.flat[np.abs(dhdt).argmax()]   > dhdt_max):
                dhdt_max = dhdt.flat[np.abs(dhdt).argmax()]
                # print('dhdt_max: ', dhdt_max)
                
            # if(dhdt.flat[np.abs(dhdt).argmax()] < (dhdt_max * 0.01)):
            #     print('Falied 2')
            #     break
                
            h = h + (dhdt * dt)
            
            for m in range(len(h)):
                for n in range(len(h[m])):
                    if(h[m][n]<0):
                        h[m][n] = 0
                        
#             if True in np.isnan(fluxX):
#                 print('NaN value found in iteration: ', r)
#                 break
                        
            if(build_plots and str(r)[-1] == '0'):
                plt.imshow(h, cmap = 'hot')
                plt.title('Height map Iteration: ' + str(r))
                plt.colorbar()
                plt.show()
                clear_output(wait = True)

    # Write T data    
    t_filename = 'temp/T_' + str(file_count) + '.csv'
    np.savetxt(t_filename, T, delimiter = ",")    
    
    # Write h data
    h_filename = 'temp/h_' + str(file_count) + '.csv'
    np.savetxt(h_filename, h, delimiter = ",") 
    
    print('File ', file_count, ' done!')
    print('Iterations done: ', r)
    
    file_count += 1

<h2>CSV -> PT</h2>    